In [1]:
#기본 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#전처리부분
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

#모델부분
from sklearn import naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier

#검증부분 - Confusion_Matrix / Preciion and Reall
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score , recall_score , f1_score

#검증부분 - Cross Validation
from sklearn.model_selection import cross_val_score #점수 나타내기

In [2]:
#파일을 읽고 필요한 정보 데이터만 가져오기
def read_file(file):
    data = pd.read_excel(file)
    text = data["text"]
    label = data['label']
    new_data = pd.DataFrame({"Text" : text,"Label" : label})
    return new_data

In [3]:
data= read_file('real6.xlsx')

In [4]:
len(data)

106448

In [5]:
def split_all_sentence(text_data): # type(text_data) => DataFrame
    result = []
    count = 0
    for idx,text in enumerate(list(text_data)): # 수정 예정
        temp = text.split(sep = " ")
        result.append(temp)
    return result

In [6]:
#data에 있는 text 데이터에 대해 url 전부 제거
def remove_url(text_data):
    result = []
    for line in text_data: 
        new_line = []
        for text in line: # 단어별로 나눠진 텍스트들이 http, pic을 담고있다면 문장에서 제외
            if ("http" not in text) and ("pic" not in text):
                new_line.append(text) 
        result.append(new_line)
    return result

## 1) 기존 코드 - 특문 제거

---
```python
#특수문자를 모두 제거하는 함수

def change(otl):
    result = []
    for line in otl:
        result_text = ""
        for word in line:
            if word.isalpha() : # 단어에 특문이 포함되지 않은 경우
                result_text += word
            else:
                for t in word: # 단어에 특문, 숫자가 포함된 경우 문자별로 파악
                    if t.isalpha():
                        result_text += t
                    else:
                        result_text += " " # 사라질 문자는 'space' 로 대체 => 5) 조정 필요
            result_text += " "
        result.append(result_text)
```
---

## 2) 기존 코드 - 공백을 없애는 함수


```python
### 기존함수 change의 공백때문에 필요했던 함수.
## 지금은 필요없음

def trim_spacing(after_remove_text):
    new_text = []
    for line in after_remove_text: # 한 줄씩 읽기.
        tmp = []
        for word in line.split(): # split(delimeter = whitespace) > default
            if word != " ":
                tmp.append(word)
        new_text.append(" ".join(tmp))
    print(new_text)
```
--- 

In [118]:
# 아래 remove_special로 대체하면 한방에 해결

def remove_special(after_remove_url): # 특수문자제거
    result = []
    for i, line in enumerate(after_remove_url):
        tmp = ' '.join([word if word.isalpha() else (''.join([w for w in word if w.isalpha()])) for word in line])
        result.append(tmp)
    return result

In [41]:
text_data = split_all_sentence(data['Text']) # 텍스트데이터만 사용

In [110]:
rurl_text = remove_url(text_data) # url 제거

In [111]:
rs_text = remove_special(rurl_text) # 특문제거

In [112]:
rs_text[0:5]

['사람들이 잘 모르는데 진짜 유명해졌으면 하는 일본 편의점 음식 일본 미니스톱 아이스 果実いちご氷 세상사람들 이것 좀 먹어보세요ㅠ유유ㅠㅠㅠ 위에는 달달한 바닐라 아이스고 밑에는 얼린 딸기에 연유가 뿌려져',
 '아이스의오락시간 아이스 의 오락시간 과연 누가 아이스의 유연성 왕일지',
 '오 오랫만에 아이스 돌체라떼 음 맛있겠다  Starbucks in 부산광역시 Busan',
 '맛집탐험대 나를 놀라게한 커피 TERAROSA TERAROSA 테라로사 날씨 냉커피 덥다 맛 맛집 사장님 서정점 아메리카노 아이스 아이스커피 양평 좋다 카페 커피 커피향 커피향기 향기 홍보',
 '로렉스 아이스 다이아고씨쥬얼리명품시계금화이트골드화이트골드']

# 2-1) Preprocessing - 형태소 추출



In [124]:
test = rs_text[0:3]

In [126]:
test

['사람들이 잘 모르는데 진짜 유명해졌으면 하는 일본 편의점 음식 일본 미니스톱 아이스 果実いちご氷 세상사람들 이것 좀 먹어보세요ㅠ유유ㅠㅠㅠ 위에는 달달한 바닐라 아이스고 밑에는 얼린 딸기에 연유가 뿌려져',
 '아이스의오락시간 아이스 의 오락시간 과연 누가 아이스의 유연성 왕일지',
 '오 오랫만에 아이스 돌체라떼 음 맛있겠다  Starbucks in 부산광역시 Busan']

In [133]:
## 출력 확인용
import MeCab

tagger = MeCab.Tagger()
real = tagger.parse(test[0])
seperated_text = real.split()[1::2] # step = 2, 짝수번째 인덱스에는 글자만 들어있음 ex) text[0] ==> '사람', [2] ==> '들', 
print(seperated_text)

['NNG,*,T,사람,*,*,*,*', 'XSN,*,T,들,*,*,*,*', 'JKS,*,F,이,*,*,*,*', 'MAG,*,T,잘,*,*,*,*', 'VV,*,F,모르,*,*,*,*', 'EC,*,F,는데,*,*,*,*', 'MAG,문장부사|양상부사,F,진짜,*,*,*,*', 'XR,*,T,유명,*,*,*,*', 'XSA+EC+VX+EP+EC,*,T,해졌으면,Inflect,XSA,EC,하/XSA/*+아/EC/*+지/VX/*+었/EP/*+으면/EC/*', 'VV,*,F,하,*,*,*,*', 'ETM,*,T,는,*,*,*,*', 'NNP,지명,T,일본,*,*,*,*', 'NNG,*,T,편의점,Compound,*,*,편의/NNG/정적사태+점/NNG/*', 'NNG,*,T,음식,*,*,*,*', 'NNP,지명,T,일본,*,*,*,*', 'NNP,*,T,미니스톱,Compound,*,*,미니/NNG/*+스톱/NNG/*', 'NNP,인명,F,아이스,*,*,*,*', 'NNG,*,F,과,*,*,*,*', 'SH,*,*,*,*,*,*,*', 'SL,*,*,*,*,*,*,*', 'SH,*,*,*,*,*,*,*', 'NNG,*,T,세상,*,*,*,*', 'NNG,*,T,사람,*,*,*,*', 'XSN,*,T,들,*,*,*,*', 'NP,*,T,이것,Inflect,NP,NP,이거/NP/*', 'MAG,성분부사|정도부사,T,좀,*,*,*,*', 'VV,*,T,먹,*,*,*,*', 'EC,*,F,어,*,*,*,*', 'VX,*,F,보,*,*,*,*', 'EP+EF,*,F,세요,Inflect,EP,EF,시/EP/*+어요/EF/*', 'UNKNOWN,*,*,*,*,*,*,*', 'NNG,*,F,위,*,*,*,*', 'JKB,*,F,에,*,*,*,*', 'JX,*,T,는,*,*,*,*', 'MAG,성분부사|양태부사,T,달달,*,*,*,*', 'XSA+ETM,*,T,한,Inflect,XSA,ETM,하/XSA/*+ᆫ/ETM/*', 'NNG,*,F,바닐라,*,*,*,*', 'NNG,*,F,

In [145]:
custom_list =  ["NNP","NNG","VV","VA","XR", "UNKNOWN"]

words = []
types = []

for i in seperated_text:
    tmp = i.split(',')
#     print(tmp)
    if tmp[0] in custom_list:
        types.append(tmp[0])
        words.append(tmp[3])
    

print(list(zip(types, words)))

[('NNG', '사람'), ('VV', '모르'), ('XR', '유명'), ('VV', '하'), ('NNP', '일본'), ('NNG', '편의점'), ('NNG', '음식'), ('NNP', '일본'), ('NNP', '미니스톱'), ('NNP', '아이스'), ('NNG', '과'), ('NNG', '세상'), ('NNG', '사람'), ('VV', '먹'), ('UNKNOWN', '*'), ('NNG', '위'), ('NNG', '바닐라'), ('NNG', '아이스'), ('NNG', '밑'), ('NNG', '딸기'), ('NNG', '연유')]


### `품사가 필요한 경우`

In [156]:
## 문장별로 실행되는 함수 => 텍스트를 for문으로 받아서 사용

def morph_extract(after_rs_text, custom_list =  ["NNP","NNG","VV","VA","XR", "UNKNOWN"]):
    tagger = MeCab.Tagger()
    real = tagger.parse(after_rs_text)
    seperated_text = real.split()[1::2] # step = 2, 짝수번째 인덱스에는 글자만 들어있음 ex) text[0] ==> '사람', [2] ==> '들', 

    words = []
    types = []

    for i in seperated_text:
        tmp = i.split(',')
        
        if tmp[0] in custom_list:
            types.append(tmp[0])
            words.append(tmp[3])

    ans = list(zip(types, words))
    
    return ans

In [158]:
morph_extract(test[2])

[('NNG', '오랫만'),
 ('NNG', '아이스'),
 ('NNG', '돌체'),
 ('NNG', '라떼'),
 ('VA', '맛있'),
 ('NNP', '부산광역시')]

### 결과값 (morph_extract(test[2]))
```python
[('NNG', '오랫만'),
 ('NNG', '아이스'),
 ('NNG', '돌체'),
 ('NNG', '라떼'),
 ('VA', '맛있'),
 ('NNP', '부산광역시')]
```

In [162]:
## 시간 매우 오래걸림. 확인용으로 5개만 사용함

after_morph_list = []
test = rs_text[0:5]

for selected_text in test:
    after_morph_list.append(morph_extract(selected_text))
    
print(len(after_morph_list))
print(after_morph_list)

5
[[('NNG', '사람'), ('VV', '모르'), ('XR', '유명'), ('VV', '하'), ('NNP', '일본'), ('NNG', '편의점'), ('NNG', '음식'), ('NNP', '일본'), ('NNP', '미니스톱'), ('NNP', '아이스'), ('NNG', '과'), ('NNG', '세상'), ('NNG', '사람'), ('VV', '먹'), ('UNKNOWN', '*'), ('NNG', '위'), ('NNG', '바닐라'), ('NNG', '아이스'), ('NNG', '밑'), ('NNG', '딸기'), ('NNG', '연유')], [('NNG', '아이스'), ('NNG', '오락'), ('NNG', '시간'), ('NNG', '아이스'), ('NNG', '오락'), ('NNG', '시간'), ('NNG', '아이스'), ('XR', '유연'), ('NNG', '왕')], [('NNG', '오랫만'), ('NNG', '아이스'), ('NNG', '돌체'), ('NNG', '라떼'), ('VA', '맛있'), ('NNP', '부산광역시')], [('NNG', '맛집'), ('NNG', '탐험대'), ('VV', '놀라'), ('NNG', '커피'), ('NNG', '테라'), ('NNG', '날씨'), ('NNG', '냉커피'), ('VA', '덥'), ('NNG', '맛'), ('NNG', '맛집'), ('NNG', '사장'), ('NNG', '서정'), ('NNG', '점'), ('NNP', '아메리카노'), ('NNP', '아이스'), ('NNG', '아이스커피'), ('NNG', '양평'), ('VA', '좋'), ('NNG', '카페'), ('NNG', '커피'), ('NNG', '커피'), ('NNG', '향'), ('NNG', '커피'), ('NNG', '향기'), ('NNG', '향기'), ('NNG', '홍보')], [('NNP', '렉스'), ('NNG', '아이스'), ('NNP', '이아고'), ('NNP

### `품사가 필요 없는 경우`, 아래의 함수로 텍스트만 가져와 사용

In [166]:
def word_extract(after_rs_text, custom_list =  ["NNP","NNG","VV","VA","XR", "UNKNOWN"]):
    tagger = MeCab.Tagger()
    real = tagger.parse(after_rs_text)
    seperated_text = real.split()[1::2] # step = 2, 짝수번째 인덱스에는 글자만 들어있음 ex) text[0] ==> '사람', [2] ==> '들', 

    words = []
    types = []

    for i in seperated_text:
        tmp = i.split(',')
        
        if tmp[0] in custom_list:
            words.append(tmp[3])
            
    return ' '.join(words)

In [168]:
after_word_extract = []

for selected_text in test:
    after_word_extract.append(word_extract(selected_text))

print(len(after_word_extract))
print(after_word_extract)

## 중간에 보이는 *은 Unknown값으로 들어감.

5
['사람 모르 유명 하 일본 편의점 음식 일본 미니스톱 아이스 과 세상 사람 먹 * 위 바닐라 아이스 밑 딸기 연유', '아이스 오락 시간 아이스 오락 시간 아이스 유연 왕', '오랫만 아이스 돌체 라떼 맛있 부산광역시', '맛집 탐험대 놀라 커피 테라 날씨 냉커피 덥 맛 맛집 사장 서정 점 아메리카노 아이스 아이스커피 양평 좋 카페 커피 커피 향 커피 향기 향기 홍보', '렉스 아이스 이아고 쥬얼리 명품 시 계금 화이트골드 화이트골드']


### semi-result

```python
5
['사람 모르 유명 하 일본 편의점 음식 일본 미니스톱 아이스 과 세상 사람 먹 * 위 바닐라 아이스 밑 딸기 연유', '아이스 오락 시간 아이스 오락 시간 아이스 유연 왕', '오랫만 아이스 돌체 라떼 맛있 부산광역시', '맛집 탐험대 놀라 커피 테라 날씨 냉커피 덥 맛 맛집 사장 서정 점 아메리카노 아이스 아이스커피 양평 좋 카페 커피 커피 향 커피 향기 향기 홍보', '렉스 아이스 이아고 쥬얼리 명품 시 계금 화이트골드 화이트골드']
```

# 2-2) Preprocessor - 자소분리 (Decompose) with `hgtk`

In [169]:
## 예시 코드

import hgtk

a = hgtk.text.decompose("아이스")
print(a.replace("ᴥ",""))
print(a)

ㅇㅏㅇㅣㅅㅡ
ㅇㅏᴥㅇㅣᴥㅅㅡᴥ


In [170]:
def decompose_text(text_list):
    new_list = []
    for text in text_list:
        decomposed = hgtk.text.decompose(text)
        new_list.append(decomposed.replace("ᴥ",""))
    return new_list

In [172]:
def remove_alpha(text_list):
    new_list = []
    alpha = "abcdefghijklmnopqrstuvwxyz"
    for text in text_list:
        one = ""
        for m in text.lower():
            if m not in alpha :
                one += m
        new_list.append(one)
    return new_list

In [175]:
d_text = decompose_text(test) # 자소분리

In [176]:
ra_text = remove_alpha(d_text) # 알파벳 제거

In [177]:
ra_text

['ㅅㅏㄹㅏㅁㄷㅡㄹㅇㅣ ㅈㅏㄹ ㅁㅗㄹㅡㄴㅡㄴㄷㅔ ㅈㅣㄴㅉㅏ ㅇㅠㅁㅕㅇㅎㅐㅈㅕㅆㅇㅡㅁㅕㄴ ㅎㅏㄴㅡㄴ ㅇㅣㄹㅂㅗㄴ ㅍㅕㄴㅇㅢㅈㅓㅁ ㅇㅡㅁㅅㅣㄱ ㅇㅣㄹㅂㅗㄴ ㅁㅣㄴㅣㅅㅡㅌㅗㅂ ㅇㅏㅇㅣㅅㅡ  ㅅㅔㅅㅏㅇㅅㅏㄹㅏㅁㄷㅡㄹ ㅇㅣㄱㅓㅅ ㅈㅗㅁ ㅁㅓㄱㅇㅓㅂㅗㅅㅔㅇㅛㅠㅇㅠㅇㅠㅠㅠㅠ ㅇㅟㅇㅔㄴㅡㄴ ㄷㅏㄹㄷㅏㄹㅎㅏㄴ ㅂㅏㄴㅣㄹㄹㅏ ㅇㅏㅇㅣㅅㅡㄱㅗ ㅁㅣㅌㅇㅔㄴㅡㄴ ㅇㅓㄹㄹㅣㄴ ㄸㅏㄹㄱㅣㅇㅔ ㅇㅕㄴㅇㅠㄱㅏ ㅃㅜㄹㅕㅈㅕ',
 'ㅇㅏㅇㅣㅅㅡㅇㅢㅇㅗㄹㅏㄱㅅㅣㄱㅏㄴ ㅇㅏㅇㅣㅅㅡ ㅇㅢ ㅇㅗㄹㅏㄱㅅㅣㄱㅏㄴ ㄱㅘㅇㅕㄴ ㄴㅜㄱㅏ ㅇㅏㅇㅣㅅㅡㅇㅢ ㅇㅠㅇㅕㄴㅅㅓㅇ ㅇㅘㅇㅇㅣㄹㅈㅣ',
 'ㅇㅗ ㅇㅗㄹㅐㅅㅁㅏㄴㅇㅔ ㅇㅏㅇㅣㅅㅡ ㄷㅗㄹㅊㅔㄹㅏㄸㅔ ㅇㅡㅁ ㅁㅏㅅㅇㅣㅆㄱㅔㅆㄷㅏ    ㅂㅜㅅㅏㄴㄱㅘㅇㅇㅕㄱㅅㅣ ',
 'ㅁㅏㅅㅈㅣㅂㅌㅏㅁㅎㅓㅁㄷㅐ ㄴㅏㄹㅡㄹ ㄴㅗㄹㄹㅏㄱㅔㅎㅏㄴ ㅋㅓㅍㅣ   ㅌㅔㄹㅏㄹㅗㅅㅏ ㄴㅏㄹㅆㅣ ㄴㅐㅇㅋㅓㅍㅣ ㄷㅓㅂㄷㅏ ㅁㅏㅅ ㅁㅏㅅㅈㅣㅂ ㅅㅏㅈㅏㅇㄴㅣㅁ ㅅㅓㅈㅓㅇㅈㅓㅁ ㅇㅏㅁㅔㄹㅣㅋㅏㄴㅗ ㅇㅏㅇㅣㅅㅡ ㅇㅏㅇㅣㅅㅡㅋㅓㅍㅣ ㅇㅑㅇㅍㅕㅇ ㅈㅗㅎㄷㅏ ㅋㅏㅍㅔ ㅋㅓㅍㅣ ㅋㅓㅍㅣㅎㅑㅇ ㅋㅓㅍㅣㅎㅑㅇㄱㅣ ㅎㅑㅇㄱㅣ ㅎㅗㅇㅂㅗ',
 'ㄹㅗㄹㅔㄱㅅㅡ ㅇㅏㅇㅣㅅㅡ ㄷㅏㅇㅣㅇㅏㄱㅗㅆㅣㅈㅠㅇㅓㄹㄹㅣㅁㅕㅇㅍㅜㅁㅅㅣㄱㅖㄱㅡㅁㅎㅘㅇㅣㅌㅡㄱㅗㄹㄷㅡㅎㅘㅇㅣㅌㅡㄱㅗㄹㄷㅡ']

---
## ready to use (end preprocess)

```python

['ㅅㅏㄹㅏㅁㄷㅡㄹㅇㅣ ㅈㅏㄹ ㅁㅗㄹㅡㄴㅡㄴㄷㅔ ㅈㅣㄴㅉㅏ ㅇㅠㅁㅕㅇㅎㅐㅈㅕㅆㅇㅡㅁㅕㄴ ㅎㅏㄴㅡㄴ ㅇㅣㄹㅂㅗㄴ ㅍㅕㄴㅇㅢㅈㅓㅁ ㅇㅡㅁㅅㅣㄱ ㅇㅣㄹㅂㅗㄴ ㅁㅣㄴㅣㅅㅡㅌㅗㅂ ㅇㅏㅇㅣㅅㅡ  ㅅㅔㅅㅏㅇㅅㅏㄹㅏㅁㄷㅡㄹ ㅇㅣㄱㅓㅅ ㅈㅗㅁ ㅁㅓㄱㅇㅓㅂㅗㅅㅔㅇㅛㅠㅇㅠㅇㅠㅠㅠㅠ ㅇㅟㅇㅔㄴㅡㄴ ㄷㅏㄹㄷㅏㄹㅎㅏㄴ ㅂㅏㄴㅣㄹㄹㅏ ㅇㅏㅇㅣㅅㅡㄱㅗ ㅁㅣㅌㅇㅔㄴㅡㄴ ㅇㅓㄹㄹㅣㄴ ㄸㅏㄹㄱㅣㅇㅔ ㅇㅕㄴㅇㅠㄱㅏ ㅃㅜㄹㅕㅈㅕ',
 'ㅇㅏㅇㅣㅅㅡㅇㅢㅇㅗㄹㅏㄱㅅㅣㄱㅏㄴ ㅇㅏㅇㅣㅅㅡ ㅇㅢ ㅇㅗㄹㅏㄱㅅㅣㄱㅏㄴ ㄱㅘㅇㅕㄴ ㄴㅜㄱㅏ ㅇㅏㅇㅣㅅㅡㅇㅢ ㅇㅠㅇㅕㄴㅅㅓㅇ ㅇㅘㅇㅇㅣㄹㅈㅣ',
 'ㅇㅗ ㅇㅗㄹㅐㅅㅁㅏㄴㅇㅔ ㅇㅏㅇㅣㅅㅡ ㄷㅗㄹㅊㅔㄹㅏㄸㅔ ㅇㅡㅁ ㅁㅏㅅㅇㅣㅆㄱㅔㅆㄷㅏ    ㅂㅜㅅㅏㄴㄱㅘㅇㅇㅕㄱㅅㅣ ',
 'ㅁㅏㅅㅈㅣㅂㅌㅏㅁㅎㅓㅁㄷㅐ ㄴㅏㄹㅡㄹ ㄴㅗㄹㄹㅏㄱㅔㅎㅏㄴ ㅋㅓㅍㅣ   ㅌㅔㄹㅏㄹㅗㅅㅏ ㄴㅏㄹㅆㅣ ㄴㅐㅇㅋㅓㅍㅣ ㄷㅓㅂㄷㅏ ㅁㅏㅅ ㅁㅏㅅㅈㅣㅂ ㅅㅏㅈㅏㅇㄴㅣㅁ ㅅㅓㅈㅓㅇㅈㅓㅁ ㅇㅏㅁㅔㄹㅣㅋㅏㄴㅗ ㅇㅏㅇㅣㅅㅡ ㅇㅏㅇㅣㅅㅡㅋㅓㅍㅣ ㅇㅑㅇㅍㅕㅇ ㅈㅗㅎㄷㅏ ㅋㅏㅍㅔ ㅋㅓㅍㅣ ㅋㅓㅍㅣㅎㅑㅇ ㅋㅓㅍㅣㅎㅑㅇㄱㅣ ㅎㅑㅇㄱㅣ ㅎㅗㅇㅂㅗ',
 'ㄹㅗㄹㅔㄱㅅㅡ ㅇㅏㅇㅣㅅㅡ ㄷㅏㅇㅣㅇㅏㄱㅗㅆㅣㅈㅠㅇㅓㄹㄹㅣㅁㅕㅇㅍㅜㅁㅅㅣㄱㅖㄱㅡㅁㅎㅘㅇㅣㅌㅡㄱㅗㄹㄷㅡㅎㅘㅇㅣㅌㅡㄱㅗㄹㄷㅡ']
```

---
## 2020.01.21 update.

**Checklist**
- 1.preprocess_func: [x]
- 2.model: []
- 3.accuracy_check: []
- 4.collect_date: []